In [1]:
import ee
import pandas as pd
import datetime

# ─── Global Limit ────────────────────────────────
HEAD_ROWS = 30  # Change this value as needed

# ─── Initialize Earth Engine ─────────────────────
try:
    ee.Initialize(project='project1-464706')
except:
    ee.Authenticate()
    ee.Initialize(project='project1-464706')

# ─── Load CSV ─────────────────────────────────────
df = pd.read_csv("/content/fixed_wkt_file(in).csv")
print("Column names:")
print(df.columns.tolist())
df['Farmer_Name'] = df['Farmer_Name'].str.strip()

# Optional: Limit to first N rows
df = df.head(HEAD_ROWS)

# ─── WKT → Geometry ──────────────────────────────
def parse_wkt(wkt_string):
    coords = wkt_string.replace("POLYGON((", "").replace("))", "")
    coords = [[float(x) for x in pair.split()] for pair in coords.split(", ")]
    return ee.Geometry.Polygon([coords])

# ─── NDVI Extraction Function ────────────────────
def get_ndvi_series(geometry):
    def compute_ndvi(img):
        ndvi = img.normalizedDifference(['B8', 'B4']).rename('NDVI')
        mean = ndvi.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=geometry,
            scale=10,
            bestEffort=True
        ).get('NDVI')
        return ee.Feature(None, {
            'system:time_start': img.get('system:time_start'),
            'NDVI_mean': mean
        })

    s2 = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") \
        .filterBounds(geometry) \
        .filterDate("2024-01-01", "2024-12-31") \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)) \
        .map(compute_ndvi)

    return s2.aggregate_array('NDVI_mean').getInfo(), \
           s2.aggregate_array('system:time_start').getInfo()

# ─── NDVI Extraction Loop ────────────────────────
records = []

for idx, row in df.iterrows():
    farmer = row['Farmer_Name']
    father = row['Father_Name']
    plot = row['Plot No']
    geometry = parse_wkt(row['WKT'])

    ndvi_vals, timestamps = get_ndvi_series(geometry)

    for ndvi, time in zip(ndvi_vals, timestamps):
        date = datetime.datetime.utcfromtimestamp(time / 1000).date()
        records.append({
            'Farmer_Name': farmer,
            'Father_Name': father,
            'Plot No': plot,
            'NDVI_Date': date,
            'NDVI': ndvi
        })

# ─── Save to CSV ─────────────────────────────────
ndvi_df = pd.DataFrame(records)
ndvi_df.to_csv("/content/ndvi_output_2024.csv", index=False)
print("Saved NDVI data for first", HEAD_ROWS, "rows.")


Column names:
['Div_Name', 'Vill_Cd', 'Vill_Name', 'Vill_Code', 'Suervisor Name', 'Farm Code', 'Farmer_Name', 'Father_Name', 'Plot No', 'Gashti No.', 'Survey Date', 'Area', 'Shar', 'Varieties', 'Crop Type', 'East', 'West', 'North', 'South', 'Lang1', 'Long1', 'Lang2', 'Long2', 'Lang3', 'Long3', 'Lang4', 'Long4', 'WKT']
Saved NDVI data for first 30 rows.
